# Install Libraries

In [1]:
!pip3 install --upgrade pip
!pip3 install numpy
!pip3 install pandas
!pip3 install requests
!pip3 install beautifulsoup4
!pip3 install selenium
!pip3 install urllib3
!brew install geckodriver

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages (20.0.2)
Updating Homebrew...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> Updated Casks
acquia-dev

To reinstall 0.26.0, run `brew reinstall geckodriver`


# Basic imports

In [2]:
import os
import multiprocessing  as mp
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import urllib.request
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

In [3]:
# Base directory. Will be used to point the 
# "chromedriver" application
ROOT_DIR = os.getcwd()

In [4]:
# Base URL that needs to be fetched
# in the website "Kita Navigator"
base_search_urlpage = 'https://www.kita-navigator.berlin.de/einrichtungen?input=Arnold-Knoblauch-Ring%2C%2014109%20Berlin%2C%20Alemanha&betb=2-2020&einfacheSuche=true&entfernung=5&lat=52.4099966&lon=13.137335'

In [5]:
# Number of pages that all 
# results are distributed.
number_of_pagination = 2

In [6]:
# List with all results of the pagination
list_all_pages_for_scrapping = []

In [7]:
# Will store all the pagination results
for page in range(1, number_of_pagination+1):
    list_all_pages_for_scrapping.append(base_search_urlpage + '&seite=' + str(page) + '&index=0')

In [8]:
# List that will store all Kitas fetched 
# for each of the pagination links
list_of_kitas_paginated = []

In [9]:
# For each paginated page, will fetch all 
# sites for each kita
for pagination_page in list_all_pages_for_scrapping:
    
    # The Chrome driver will open a page to load 
    # the page and all JavaScript objects
    webdriver_path = ROOT_DIR + "/chromedriver"
    driver = Chrome(webdriver_path)

    # Will open the webpage and load
    # all objects in a separate window
    driver.get(pagination_page)

    # Execute script to scroll down the page to 
    # load all JavaScript objects
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")

    # The window will open for 2 seconds to give
    # time to load all objects
    time.sleep(2)

    # All elements in the HTML 
    # that contains a link "href" will be fetched and 
    # stored in the "list_of_kitas_paginated" list
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        list_of_kitas_paginated.append(elem.get_attribute("href"))

    # Those 2 list comprehensions will prune out
    # URLs that aren't related with Kitas (I checked the 
    # pattern by myself)

    # Condition #1: Has the "https://www.kita-navigator.berlin.de/einrichtungen/" string
    list_of_kitas_paginated = \
        [i for i in list_of_kitas_paginated if 'https://www.kita-navigator.berlin.de/einrichtungen/' in i]

    # Condition #2: Has the "einfacheSuche=true" string
    list_of_kitas_paginated = \
        [i for i in list_of_kitas_paginated if 'einfacheSuche=true' in i]

In [10]:
# As we have some duplicated values
# I'll transform in a "set" data type to prune out the 
# duplicated results
list_of_kitas_paginated = set(list_of_kitas_paginated)

In [11]:
# Put all info in a Pandas DataFrame
# and adjust the column name
df_kitas_url = pd.DataFrame(list_of_kitas_paginated)
df_kitas_url.columns = ['kita_url']

In [12]:
# Check (Must have 15, as the original search)
df_kitas_url

,kita_url
0,https://www.kita-navigator.berlin.de/einrichtu...
1,https://www.kita-navigator.berlin.de/einrichtu...
2,https://www.kita-navigator.berlin.de/einrichtu...
3,https://www.kita-navigator.berlin.de/einrichtu...
4,https://www.kita-navigator.berlin.de/einrichtu...
5,https://www.kita-navigator.berlin.de/einrichtu...
6,https://www.kita-navigator.berlin.de/einrichtu...
7,https://www.kita-navigator.berlin.de/einrichtu...
8,https://www.kita-navigator.berlin.de/einrichtu...
9,https://www.kita-navigator.berlin.de/einrichtu...


In [13]:
# Save in a CSV file to be consumed 
# in another notebook
df_kitas_url.to_csv('df_kitas_url.csv', index=None)